In [1]:
from datetime import datetime
import json
import os
import re
from pathlib import Path
from typing import Dict, List
from urllib.parse import parse_qs
from bs4 import BeautifulSoup, Tag
from functional import seq
import scrapy
from scrapy import signals
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from scrapy.http import HtmlResponse
from scrapy_playwright.page import PageMethod
from playwright.sync_api import sync_playwright, Playwright
import requests
import asyncio
from functional import seq
from typing import Optional
from playwright.async_api import async_playwright, Playwright


In [2]:
link= "https://www.ox.ac.uk/admissions/graduate/courses/msc-applied-linguistics-and-second-language-acquisition"
response= requests.get(link)
soup= BeautifulSoup(response.content, "html.parser")


In [6]:
modules=soup.select("#content-tab ul li")
for module in modules:
    if "core" in module.find_previous('p').text.lower():
        print(module.text)
    elif "option" in module.find_previous('p').text.lower():
        print(module.text)

Foundations of SLA Theory
First Language Acquisition & Bilingualism
Vocabulary Acquisition
Communication of Meaning
Sociolinguistics
Individual and Group Differences


In [4]:
programme_duration = soup.find("dt", text="Programme duration")
durations = programme_duration.find_next_sibling().get_text(separator="|", strip=True)
study_mode_duration_map = {}

for duration_mode in durations.split('|'):
    if 'full-time' in duration_mode:
        study_mode_duration_map['full-time'] = duration_mode.split('full-time')[0].strip()
    elif 'part-time' in duration_mode:
        study_mode_duration_map['part-time'] = duration_mode.split('part-time')[0].strip()
print(study_mode_duration_map)
if study_mode.lower() in study_mode_duration_map:
    duration = study_mode_duration_map[study_mode.lower()]
else:
    duration = None
            

{'part-time': 'MSc: Two years'}


In [53]:
link= "https://www.bris.ac.uk/unit-programme-catalogue/RouteStructure.jsa?_ga=2.171731944.932948495.1615801524-674141097.1605255911&ayrCode=24%2F25&byCohort=N&programmeCode=9LAWD006T&_gl=1*11qmqnt*_ga*MTE3MjA4Mjg1MC4xNzE3MDA2MzI4*_ga_6R8SPL3HLT*MTcxNzAyMDUyMS4zLjEuMTcxNzAyMTE1MC42MC4wLjA."
response= requests.get(link)
soup= BeautifulSoup(response.content, "html.parser")

# playwright = await async_playwright().start()
# browser = await playwright.chromium.launch(headless = True)
# page = await browser.new_page()

# await page.goto("https://www.bristol.ac.uk/study/language-requirements/profile-a/")
# content = await page.content()

# await browser.close()
# await playwright.stop()
# soup = BeautifulSoup(content, "html.parser")


In [54]:
modules = []
table = soup.select_one('.table-basic')
table.select_one('thead').decompose()

for row in table.select('tr'):
    tds = row.select('td')
    if not tds[0].select_one('a'):
        continue

    unit_name, _, _, status, _ = tds

    link = f"https://www.bristol.ac.uk{unit_name.select_one('a')['href']}"
    title = unit_name.select_one('a').text.strip()

    modules.append({
        'type': status.text.strip(),
        'title': title,
        'link': link
    })
ok=soup.select("#uobcms-content  div.column.grid_8 li")
for i in ok:
    try:
        linx=i.select_one("a")["href"]
        linx= f'https://www.bris.ac.uk/{linx}'
        print(linx)
    except:
        continue
    response= requests.get(linx)
    soup= BeautifulSoup(response.content, "html.parser")
    for row in table.select('tr'):
        tds = row.select('td')
        if not tds[0].select_one('a'):
            continue

        unit_name, _, _, status, _ = tds

        link = f"https://www.bristol.ac.uk{unit_name.select_one('a')['href']}"
        title = unit_name.select_one('a').text.strip()

        modules.append({
            'type': status.text.strip(),
            'title': title,
            'link': link
        })


23
https://www.bris.ac.uk//unit-programme-catalogue/RouteStructure.jsa;jsessionid=68DA00335113F64427DB4B6C368E372D?byCohort=N&cohort=Y&routeLevelCode=1&ayrCode=24%2F25&modeOfStudyCode=Part+Time&programmeCode=9LAWD006T
https://www.bris.ac.uk//unit-programme-catalogue/RouteStructure.jsa;jsessionid=68DA00335113F64427DB4B6C368E372D?byCohort=N&cohort=Y&routeLevelCode=2&ayrCode=24%2F25&modeOfStudyCode=Part+Time&programmeCode=9LAWD006T
69


In [29]:
link="https://www.leeds.ac.uk/language-centre/doc/english-language-certificates"
response= requests.get(link)
soup= BeautifulSoup(response.content, "html.parser")

In [33]:
scores = []
certificates = {}
toefl = soup.find('h3', text=re.compile("TOEFL iBT"))\
    .find_next()\
    .find_next()
pearson_test_of_english = soup.find(
    'h3',
    text=re.compile("Pearson Test of English"))\
    .find_next()
languagecert= soup.find('h3', text=re.compile("LanguageCert Academic"))\
    .find_next()

table_headers = toefl.select('th')[1:]
for table_header in table_headers:
    content = table_header.text.split(' ')[-1].strip()
    if content not in certificates:
        certificates[content] = {}
        scores.append(content)

for (name, table) in [('TOEFL', toefl),
                        ('PTE', pearson_test_of_english),
                        ('LanguageCert',languagecert)]:
    rows = table.select('tr')
    for row in rows[1:]:
        cols = row.select('td')
        skill = cols[0].text.split(' ')[0].lower().replace('\xa0', '')
        for i, col in enumerate(cols):
            if name not in certificates[scores[i-1]]:
                certificates[scores[i-1]][name] = {
                        skill: col.text.strip()}
            else:
                certificates[scores[i-1]][name][skill] = col.text\
                                                            .strip()


print(certificates)




{'4.0': {'TOEFL': {'overall': '50', 'reading': '9', 'writing': '13', 'speaking': '16', 'listening': '12'}, 'PTE': {'overall': '43', 'reading': '43', 'writing': '43', 'speaking': '43', 'listening': '43'}, 'LanguageCert': {'overall': '45', 'reading': '45', 'writing': '45', 'speaking': '45', 'listening': '45'}}, '4.5': {'TOEFL': {'overall': '57', 'reading': '12', 'writing': '15', 'speaking': '17', 'listening': '13'}, 'PTE': {'overall': '48', 'reading': '48', 'writing': '48', 'speaking': '48', 'listening': '48'}, 'LanguageCert': {'overall': '50', 'reading': '50', 'writing': '50', 'speaking': '50', 'listening': '50'}}, '5.0': {'TOEFL': {'overall': '63', 'reading': '14', 'writing': '16', 'speaking': '19', 'listening': '14'}, 'PTE': {'overall': '54', 'reading': '54', 'writing': '54', 'speaking': '54', 'listening': '54'}, 'LanguageCert': {'overall': '55', 'reading': '55', 'writing': '55', 'speaking': '55', 'listening': '55'}}, '5.5': {'TOEFL': {'overall': '72', 'reading': '18', 'writing': '19'

In [35]:
language_requirements = []
for cert in certificates['7.0'].keys():
    print(cert, certificates['7.0'][cert])
    scores = []
    for score in certificates['7.0'][cert].keys():
        scores.append(f'{score}: {certificates["6.5"][cert][score]}')
    print(', '.join(scores))

    language_requirements.append({
        "language": "English",
        "test": cert,
        "score": ', '.join(scores)
    })
print(language_requirements)

TOEFL {'overall': '95', 'reading': '23', 'writing': '24', 'speaking': '25', 'listening': '23'}
overall: 88, reading: 22, writing: 22, speaking: 24, listening: 22
PTE {'overall': '68', 'reading': '68', 'writing': '68', 'speaking': '68', 'listening': '68'}
overall: 64, reading: 64, writing: 64, speaking: 64, listening: 64
LanguageCert {'overall': '75', 'reading': '75', 'writing': '75', 'speaking': '75', 'listening': '75'}
overall: 70, reading: 70, writing: 70, speaking: 70, listening: 70
[{'language': 'English', 'test': 'TOEFL', 'score': 'overall: 88, reading: 22, writing: 22, speaking: 24, listening: 22'}, {'language': 'English', 'test': 'PTE', 'score': 'overall: 64, reading: 64, writing: 64, speaking: 64, listening: 64'}, {'language': 'English', 'test': 'LanguageCert', 'score': 'overall: 70, reading: 70, writing: 70, speaking: 70, listening: 70'}]


In [38]:
ok="Clinical Education MEd "
qualification=ok.split(" ")[-1]
if(qualification=="(Eng)"):
    qualification=ok.split(" ")[-2]
if(qualification=="Certificate"):
    qualification="Postgraduate Certificate"
if(qualification=="Diploma"):
    qualification="Postgraduate Diploma"
print(qualification)

In [45]:
language_certificates= {}
i=0
for i in soup.select("table"):
    if(i==0):
        test="Pearson Test of English (PTE) Academic"
        language_certificates.append({"language":"English","test": test,})
    elif(i==1):
        test="TOEFL iBT (Test of English as a Foreign Language Internet-Based Test)"
    elif(i==2):
        test="LanguageCert Academic"
    

<table>
<thead>
<tr>
<th>Your Pearson Test of English (PTE) Academic score </th>
<th>Equivalent to IELTS 4.0 </th>
<th>Equivalent to IELTS 4.5 </th>
<th>Equivalent to IELTS 5.0 </th>
<th>Equivalent to IELTS 5.5 </th>
<th>Equivalent to IELTS 6.0</th>
<th>Equivalent to IELTS 6.5 </th>
<th> Equivalent to IELTS 7.0 </th>
<th>Equivalent to IELTS 7.5 </th>
</tr>
</thead>
<tbody>
<tr>
<td>Overall </td>
<td>43</td>
<td>48</td>
<td>54</td>
<td>59</td>
<td>60</td>
<td>64</td>
<td>68</td>
<td>74</td>
</tr>
<tr>
<td>Reading component </td>
<td>43</td>
<td>48</td>
<td>54</td>
<td>59</td>
<td>60</td>
<td>64</td>
<td>68</td>
<td>74</td>
</tr>
<tr>
<td>Writing component </td>
<td>43</td>
<td>48</td>
<td>54</td>
<td>59</td>
<td>60</td>
<td>64</td>
<td>68</td>
<td>74</td>
</tr>
<tr>
<td>Speaking component </td>
<td>43</td>
<td>48</td>
<td>54</td>
<td>59</td>
<td>60</td>
<td>64</td>
<td>68</td>
<td>74</td>
</tr>
<tr>
<td>Listening component </td>
<td>43</td>
<td>48</td>
<td>54</td>
<td>59</td>
<td>60</td

In [39]:
start_dates = []
selector=soup.select('#main .uol-key-facts dt')
for i in selector:
    if(i.text.lower()=="start date"):
        date=i.findNext('dd').text
        start_dates.append(date)
    elif(i.text.lower()=="start dates"):
        try:
            for j in i.findNextSiblings('dd'):
                start_dates.append(j.text)
        except:
            pass
print(start_dates)

['March 2024', 'May 2024', 'July 2024']


In [40]:
link= "https://courses.leeds.ac.uk/f360/advanced-mechanical-engineering-msc-eng-"
response= requests.get(link)
soup= BeautifulSoup(response.content, "html.parser")

In [43]:
tuitions=[]
durations="12 months full time"
for duration in durations.split(','):
    if "full time" in duration:
        study_mode="Full-time"
    elif "part time" in duration:
        study_mode="Part-time"
    selector=soup.select('#main .uol-key-facts dt')
    for i in selector:
        if(i.text.lower()=="uk fees"):
            student_category="UK"
            fee=i.findNext('dd').text
            tuitions.append({"student_category":student_category, "study_mode":study_mode, "duration":duration, "tuition":fee})
        elif(i.text.lower()=="international fees"):
            student_category="International"
            fee=i.findNext('dd').text
            tuitions.append({"student_category":student_category, "study_mode":study_mode, "duration":duration, "tuition":fee})
print(tuitions)
    
    

[{'student_category': 'UK', 'study_mode': 'Full-time', 'duration': '12 months full time', 'tuition': '£13,750 (Total)'}, {'student_category': 'International', 'study_mode': 'Full-time', 'duration': '12 months full time', 'tuition': '£31,000 (Total)'}]


In [ ]:
ok= soup.select('h2 a')
for i in ok:
    link= i.get('href')
    print(link)
    title= i.text
    print(title)
    qualification=i.text.split(" ")[-1]
    if(qualification=="(Eng)"):
        qualification=i.text.split(" ")[-2]
    if(qualification=="Certificate"):
        qualification="Postgraduate Certificate"
    if(qualification=="Diploma"):
        qualification="Postgraduate Diploma"
    print(qualification)
    durations= i.findNext('dl').text.replace("Duration","").strip()
    print(durations)



In [13]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

await page.goto("https://webprod3.leeds.ac.uk/catalogue/modulesearch.asp?L=TP&Y=202425&E=all&N=all&S=+&A=any")
content = await page.content()

await browser.close()
await playwright.stop()
soup = BeautifulSoup(content, "html.parser")

In [14]:
#leeds_module_link
modules_link_map= {}
tables = soup.find_all('table', {'width': '100%'})
for table in tables:
    for row in table.find_all('tr'):
        name= row.select('td')[3].text.strip().lower()
        link= row.select('td')[2].find('a')['href']
        link= f'https://webprod3.leeds.ac.uk/catalogue/{link}'
        modules_link_map[name]= link


In [31]:
link= 'https://courses.leeds.ac.uk/j634/teaching-english-for-academic-purposes-ma#content'
response= requests.get(link)
soup= BeautifulSoup(response.content, 'html.parser')

In [35]:
modules=[]
try:
    modules=[]
    selector=soup.select('#section-content  table tr')
    for i in selector:
        if(i.select('th')):
            continue
        title= i.select('td')[0].text.strip()
        type_text=i.find_previous('h4').text.lower().strip()
        if(type_text.find("optional")!=-1):
            type="Optional"
        else:
            type="Compulsory"
        try:
            link= modules_link_map[title.lower().strip()]
        except:
            link= ""
        modules.append({"title":title, "type":type,"link":link})
except:
    pass
try:
    language=soup.select_one('h1.page-heading__title')
    if(language.text.lower().find("language for")!=-1):
        title=language.text.strip()
        type="Compulsory"
        link=""
        modules.append({"title":title, "type":type,"link":link})
except:
    pass
try:
    selector=soup.select('#section-content div p strong')
    for i in selector:
        title= i.text.strip()
        if(title=="Optional modules" or title=="Compulsory modules"):
            continue
        type_text=i.find_previous('h3').text.lower().strip()
        if(type_text.find("optional")!=-1):
            type="Optional"
        else:
            type="Compulsory"
        try:
            finder= title.split("(")[0].strip().lower()
            link= modules_link_map[finder]
        except:
            link= ""
        modules.append({"title":title, "type":type,"link":link})
except:
    pass
try:
    selector=soup.select('#section-content div li')
    for i in selector:
        title= i.text.split("-")[0].strip()
        if(title=="Optional modules" or title=="Compulsory modules"):
            continue
        try:
            type_text=i.find_previous('h3').text.lower().strip()
        except:
            type_text=i.find_previous('strong').text.lower().strip()
        if(type_text.find("optional")!=-1):
            type="Optional"
        else:
            type="Compulsory"
        try:
            finder= title.split("(")[0].strip().lower()
            link= modules_link_map[finder]
        except:
            link= ""
        modules.append({"title":title, "type":type,"link":link})
except:
    pass

print(modules)



[{'title': 'Participants, Practices and Pedagogies in Academic Communication', 'type': 'Compulsory', 'link': 'https://webprod3.leeds.ac.uk/catalogue/dynmodules.asp?Y=202425&M=ELU-5001M'}, {'title': 'Knowledge, Politics and Cultures in Academia', 'type': 'Compulsory', 'link': 'https://webprod3.leeds.ac.uk/catalogue/dynmodules.asp?Y=202425&M=ELU-5002M'}, {'title': 'Text Analysis', 'type': 'Compulsory', 'link': 'https://webprod3.leeds.ac.uk/catalogue/dynmodules.asp?Y=202425&M=ELU-5003M'}, {'title': 'Dissertation or Portfolio', 'type': 'Compulsory', 'link': ''}, {'title': 'Principles of Curriculum and Assessment Design', 'type': 'Optional', 'link': 'https://webprod3.leeds.ac.uk/catalogue/dynmodules.asp?Y=202425&M=ELU-5004M'}, {'title': 'Practioner Development and Scholarship', 'type': 'Optional', 'link': ''}, {'title': 'Academic Leadership', 'type': 'Optional', 'link': 'https://webprod3.leeds.ac.uk/catalogue/dynmodules.asp?Y=202425&M=ELU-5006M'}, {'title': 'Ethics and Malpractice in HE', '

In [34]:
selector=soup.select('#section-content div li')
for i in selector:
    title= i.text.split("-")[0].strip()
    if(title=="Optional modules" or title=="Compulsory modules"):
        continue
    type_text=i.find_previous('h3').text.lower().strip()
    if(type_text.find("optional")!=-1):
        type="Optional"
    else:
        type="Compulsory"
    try:
        finder= title.split("(")[0].strip().lower()
        link= modules_link_map[finder]
    except:
        link= ""
    modules.append({"title":title, "type":type,"link":link})

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
table = soup.find_all('table', {'width': '100%'})
print(table)

In [12]:
link= 'https://warwick.ac.uk/services/finance/studentfinance/fees/postgraduatefees/'
response= requests.get(link)
soup= BeautifulSoup(response.content, 'html.parser')

In [13]:
soup = BeautifulSoup(
                    response.content,
                    'html.parser',
                    from_encoding='utf-8')
tuition_fees = {}
tuition_fees_table = soup.find(attrs={"id": "searchable-table"}) \
                            .find('tbody')
for row in tuition_fees_table.select('tr'):
    cells = row.select('td')
    course_name = cells[0].text.strip()
    study_mode = cells[2].text.strip()
    student_category = cells[3].text.strip()
    fee = cells[6].text
    tuition_fee_item = {
        "study_mode": study_mode,
        "duration": '',
        "student_category": student_category,
        "fee": fee
    }
    if course_name in tuition_fees:
        tuition_fees[course_name].append(tuition_fee_item)
    else:
        tuition_fees[course_name] = [tuition_fee_item]
print(tuition_fees)

{'Accounting and Finance (MSc)': [{'study_mode': 'Full Time', 'duration': '', 'student_category': 'Home', 'fee': '£33,200'}, {'study_mode': 'Full Time', 'duration': '', 'student_category': 'Overseas', 'fee': '£40,700'}], 'Advanced Legal Studies (LLM)': [{'study_mode': 'Full Time', 'duration': '', 'student_category': 'Home', 'fee': '£11,060'}, {'study_mode': 'Full Time', 'duration': '', 'student_category': 'Overseas', 'fee': '£22,560'}, {'study_mode': 'Part Time', 'duration': '', 'student_category': 'Home', 'fee': '£5,530'}, {'study_mode': 'Part Time', 'duration': '', 'student_category': 'Overseas', 'fee': '£11,280'}], 'Advanced Management (Postgraduate Diploma)': [{'study_mode': 'Part Time', 'duration': '', 'student_category': 'ALL', 'fee': '£18,550'}], 'Advanced Mechanical Engineering (MSc)': [{'study_mode': 'Full Time', 'duration': '', 'student_category': 'Home', 'fee': '£11,600'}, {'study_mode': 'Full Time', 'duration': '', 'student_category': 'Overseas', 'fee': '£28,190'}], 'Analyt

In [15]:
url= "https://warwick.ac.uk/study/postgraduate/courses/pga-pgcert-pgdip-behavioural-science"
response= requests.get(url)
soup= BeautifulSoup(response.content, 'html.parser')
ok=soup.find("h3",string="Fees and funding")
ok=ok.next_sibling.next_element.text
#fee=£18,555
fee= re.search(r'£(\d+,\d+)',ok).group(0)
print(fee)
print(ok)

£18,550
The 2024 fees for the Executive Diploma in Behavioural Science is £18,550. There are also a number of scholarships and bursaries available for candidates enrolling on the Warwick Executive Diplomas in 2024.


In [2]:
link= 'https://warwick.ac.uk/study/postgraduate/apply/english'
response= requests.get(link)
soup= BeautifulSoup(response.content, 'html.parser')
ok=soup.select_one('div.column-2 strong').text
date_pattern = re.compile(r'\d{1,2} \w+ \d{4}')
date = date_pattern.search(ok).group(0)
print(date)

2 August 2024


In [8]:
def _get_tuitions(soup: BeautifulSoup) -> list:
    try:
        tuitions = []

        duration_dict =_get_duration(soup)
        if not duration_dict:
            return []

        for student_category in ['uk', 'international']:
            fees_section = seq(soup.select('#fees h3')).find(lambda x: student_category in x.text.lower())
            if not fees_section:
                continue
            fees_section = fees_section.next_sibling.next_sibling
            delimiter = ';' if ';' in fees_section.text else '\n'
            for fee in fees_section.text.split(delimiter):
                for study_mode, duration in duration_dict.items():
                    if study_mode in fee:
                        tuitions.append({
                            'study_mode': study_mode,
                            'duration': duration,
                            'student_category': student_category,
                            'fee': fee.replace(study_mode, '').strip()
                        })
        if tuitions == []:
            for study_mode, duration in duration_dict.items():
                tuitions.append({
                                'study_mode': study_mode,
                                'duration': duration,
                                'student_category': '',
                                'fee': ''
                            })
    except AttributeError:
        tuitions = []
    return tuitions
ok= _get_tuitions(soup)
print(ok)


[]


In [12]:
def _get_entry_requirements(json_data: dict) -> Optional[str]:
    try:
        try:
            basic_requirements = json_data['routing']['entry']['entryRequirement']['pgtDefault']
            other_requirements = json_data['routing']['entry']['otherRequirements']

            entry_requirements = "".join(seq([basic_requirements, other_requirements]).filter(lambda x: x).to_list())
        except:
            entry_requirements = json_data['routing']['entry']['entryRequirement']
        try:
            entry_requirements= json_data['routing']['entry']['otherRequirements']
        except:
            pass
    
    except (AttributeError, TypeError):
        entry_requirements = ""
    return entry_requirements
entry_requirements= _get_entry_requirements(data)
print(entry_requirements)

<p>The normal entry requirement for the BSc is a Diploma in Higher Education (120 Level 5 credits) (or overseas equivalent). Applicants must be registered nurses with the NMC or an equivalent governing body for international applicants. IELTS 6.5 overall, minimum of 6.0 in all skills.</p>
<p>Applicants must be qualified nurses with a current registration either in the UK or home country. Applicants will have a diploma in nursing that has been awarded by a recognised academic institution.</p>
<p>Visit our admissions webpages to view our <a href="https://www.kcl.ac.uk/study/postgraduate-taught/how-to-apply/entry-requirements/english-language-requirements">English language entry requirements</a></p>


In [15]:
import requests

cookies = {
    'OptanonAlertBoxClosed': '2024-03-28T08:51:00.450Z',
    '_gid': 'GA1.3.509480863.1711833552',
    '_fbp': 'fb.2.1711902939866.201974845',
    '__hs_cookie_cat_pref': '1:true_2:true_3:true',
    'hubspotutk': '338f7eef2923d6365b019b9516e5b2ec',
    '__hssrc': '1',
    '_gcl_au': '1.1.1626541155.1711902948',
    '_hjSessionUser_3882840': 'eyJpZCI6IjNjZDAxM2Y3LWU1YjEtNTBlZC1hZjUzLWI4YTFmODJiMDk3NiIsImNyZWF0ZWQiOjE3MTE5MDI5NDExNzYsImV4aXN0aW5nIjp0cnVlfQ==',
    '__hstc': '102787167.338f7eef2923d6365b019b9516e5b2ec.1711902945790.1711914310426.1711924218847.3',
    '_ga_2B2LNW4G7M': 'GS1.1.1711947718.6.0.1711947718.0.0.0',
    '_ga_MB7LPD0MRQ': 'GS1.1.1711950710.7.1.1711952107.0.0.0',
    '_ga': 'GA1.3.1650481729.1711615941',
    '_gat_gtag_UA_228896_1': '1',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Mon+Apr+01+2024+13%3A51%3A53+GMT%2B0600+(Bangladesh+Standard+Time)&version=202402.1.0&browserGpcFlag=0&isIABGlobal=false&consentId=1aec5d0b-87c7-45e8-8e05-b61c16202a81&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=H105%3A1%2CH148%3A1%2CH103%3A1%2CH130%3A1%2CH24%3A1%2CH104%3A1%2CH1%3A1%2CH201%3A1%2CH179%3A1%2CH19%3A1%2CH20%3A1%2CH144%3A1%2CH181%3A1%2CH117%3A1%2CH33%3A1%2CH48%3A1%2CH195%3A1%2CH9%3A1%2CH11%3A1%2CH106%3A1%2CH204%3A1%2CH57%3A1%2CH58%3A1%2CH59%3A1%2CH202%3A1%2CH21%3A1%2CH114%3A1%2CH115%3A1%2CH116%3A1%2CH23%3A1%2CH149%3A1%2CH173%3A1%2CH203%3A1%2CH150%3A1%2CH185%3A1%2CH126%3A1%2CH161%3A1%2CH127%3A1%2CH156%3A1%2CH128%3A1%2CH140%3A1%2CH198%3A1%2CH180%3A1%2CH62%3A1%2CH200%3A1%2CH187%3A1%2CH25%3A1%2CH26%3A1&genVendors=&geolocation=BD%3BC&AwaitingReconsent=false',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'accesstoken': 'JjV9NgvYm8BQgTNx2AtThsRBeK5qZxArDnRc2SKrzYWzvsS6',
    # 'cookie': 'OptanonAlertBoxClosed=2024-03-28T08:51:00.450Z; _gid=GA1.3.509480863.1711833552; _fbp=fb.2.1711902939866.201974845; __hs_cookie_cat_pref=1:true_2:true_3:true; hubspotutk=338f7eef2923d6365b019b9516e5b2ec; __hssrc=1; _gcl_au=1.1.1626541155.1711902948; _hjSessionUser_3882840=eyJpZCI6IjNjZDAxM2Y3LWU1YjEtNTBlZC1hZjUzLWI4YTFmODJiMDk3NiIsImNyZWF0ZWQiOjE3MTE5MDI5NDExNzYsImV4aXN0aW5nIjp0cnVlfQ==; __hstc=102787167.338f7eef2923d6365b019b9516e5b2ec.1711902945790.1711914310426.1711924218847.3; _ga_2B2LNW4G7M=GS1.1.1711947718.6.0.1711947718.0.0.0; _ga_MB7LPD0MRQ=GS1.1.1711950710.7.1.1711952107.0.0.0; _ga=GA1.3.1650481729.1711615941; _gat_gtag_UA_228896_1=1; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Apr+01+2024+13%3A51%3A53+GMT%2B0600+(Bangladesh+Standard+Time)&version=202402.1.0&browserGpcFlag=0&isIABGlobal=false&consentId=1aec5d0b-87c7-45e8-8e05-b61c16202a81&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=H105%3A1%2CH148%3A1%2CH103%3A1%2CH130%3A1%2CH24%3A1%2CH104%3A1%2CH1%3A1%2CH201%3A1%2CH179%3A1%2CH19%3A1%2CH20%3A1%2CH144%3A1%2CH181%3A1%2CH117%3A1%2CH33%3A1%2CH48%3A1%2CH195%3A1%2CH9%3A1%2CH11%3A1%2CH106%3A1%2CH204%3A1%2CH57%3A1%2CH58%3A1%2CH59%3A1%2CH202%3A1%2CH21%3A1%2CH114%3A1%2CH115%3A1%2CH116%3A1%2CH23%3A1%2CH149%3A1%2CH173%3A1%2CH203%3A1%2CH150%3A1%2CH185%3A1%2CH126%3A1%2CH161%3A1%2CH127%3A1%2CH156%3A1%2CH128%3A1%2CH140%3A1%2CH198%3A1%2CH180%3A1%2CH62%3A1%2CH200%3A1%2CH187%3A1%2CH25%3A1%2CH26%3A1&genVendors=&geolocation=BD%3BC&AwaitingReconsent=false',
    'dnt': '1',
    'referer': 'https://www.kcl.ac.uk/study/postgraduate-taught/courses/applied-neuroscience-msc',
    'sec-ch-ua': '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
}

params = {
    'linkDepth': '1',
}

response = requests.get(
    'https://www.kcl.ac.uk/api/delivery/projects/website/entries/72e8ecb1-123e-4621-bfef-0cbf249eb40e',
    params=params,
    cookies=cookies,
    headers=headers,
)
data2=response.json()
with open('data2.json', 'w') as f:
    json.dump(data2, f, indent=4)

In [108]:
try:
    mapper = {
        'ukFee': ('uk', 'Full-time'),
        'internationalFee': ('international', 'Full-time'),
        'ukPartTimeFee': ('uk', 'Part-time'),
        'internationalPartTimeFee': ('international', 'Part-Time')
    }

    # # TODO: extract durations
    tuitions = []
    try:
        for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']):
            fee = json_data['routing']['entry'][k]
            duration = json_data['routing']['entry']['duration']
            if not fee:
                continue

            fee = re.findall(r'£\d+,\d+', fee).pop(0)

            student_category, study_mode = mapper[k]
            tuitions.append({
                'study_mode': study_mode,
                'duration': duration,
                'student_category': student_category,
                'fee': fee
            })
    except:
        pass
    if(len(tuitions) == 0):
        try:
            for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']['relatedCourses'][0]['relatedCourses'][0]):
                fee=json_data['routing'] ['entry'] ['relatedCourses'][0]['relatedCourses'][0][k]
                duration= json_data['routing']['entry']['relatedCourses'][0]['relatedCourses'][0]['duration']

                if not fee:
                    continue

                fee = re.findall(r'£\d+,\d+', fee).pop(0)
        
                student_category, study_mode = mapper[k]
                tuitions.append({
                    'study_mode': study_mode,
                    'duration': duration,
                    'student_category': student_category,
                    'fee': fee
                })
        except:
            pass
    if(len(tuitions) == 0):
        try:
            for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']):
                fee = json_data['routing']['entry']["furtherFeeInformation"]
                duration = json_data['routing']['entry']['duration']
                if not fee:
                    continue

                fee = re.findall(r'£\d+,\d+', fee).pop(0)

                student_category, study_mode = mapper[k]
                tuitions.append({
                    'study_mode': study_mode,
                    'duration': duration,
                    'student_category': student_category,
                    'fee': fee
                })
        except:
            pass
    if(len(tuitions) == 0):
        try:
            for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']):
                fee = json_data['routing']['entry']["onlineCourseFeeInformation"]
                duration = json_data['routing']['entry']['duration']
                if not fee:
                    continue

                fee = re.findall(r'&pound;(\d+,\d+)', fee).pop(0)

                student_category, study_mode = mapper[k]
                tuitions.append({
                    'study_mode': study_mode,
                    'duration': duration,
                    'student_category': student_category,
                    'fee': f'£{fee}'


                })
        except:
            pass
        
    
except IndexError:
    tuitions = []

return tuitions


[{'study_mode': 'Full-time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'uk', 'fee': '£1,654'}, {'study_mode': 'Full-time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'international', 'fee': '£1,654'}, {'study_mode': 'Part-time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'uk', 'fee': '£1,654'}, {'study_mode': 'Part-Time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'international', 'fee': '£1,654'}]


Playwright Section
---Use if needed---

In [22]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

await page.goto("https://www.sheffield.ac.uk/postgraduate/taught/courses/2024/advanced-clinical-practice-gp-mmedsci")
content = await page.content()

await browser.close()
await playwright.stop()
soup = BeautifulSoup(content, "html.parser")

In [11]:
text = "1 year, full-time\n2–3 years part-time\n 2/3 years"

# Define the regex pattern to match the previous word before "year" or "years"
pattern = r'(?<=\b)\S+\s+(?=year|years)'

# Search for the pattern in the text
matches = re.findall(pattern, text)
print(matches)

['1 ', '2–3 ', '2/3 ']
